# AB 1568 Data Research 

- What happened to the money dedicated to CRAs, approximately $1.7b?
- If EIFDs raised sales taxes how likely are eligible voters to support MORE taxes? (Stage whisper: how much more sales tax are folks willing to pay? L.A. Co. is at 9.75%) **Effect of Sales tax on growth/housing?** 
- How big are EIFD-eligible areas in the most affected counties. How many people need to sign up? 
- Percentage of counties eligible for EIFD 
- Estimate effects of rent on sample EIFD 
- Possible amount raised for highest EIFDS 

In [110]:
import pandas as pd
from bokeh.charts import Line, output_file, show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

* EFID-elidblge are in SB 375 Infill locations. 

```html
21061.3.  "Infill site" means a site in an urbanized area that
meets either of the following criteria:
   (a) The site has not been previously developed for urban uses and
both of the following apply:
       (1) The site is immediately adjacent to parcels that are developed
with qualified urban uses, or at least 75 percent of the perimeter
of the site adjoins parcels that are developed with qualified urban
uses, and the remaining 25 percent of the site adjoins parcels that
have previously been developed for qualified urban uses.
       (2) No parcel within the site has been created within the past 10
years unless the parcel was created as a result of the plan of a
redevelopment agency.
   (b) The site has been previously developed for qualified urban
uses.
```

## Source Data
* [Parcels](http://egis3.lacounty.gov/dataportal/2015/09/11/california-statewide-parcel-boundaries/)
* [LA County Parcels](https://data.lacounty.gov/Parcel-/Assessor-Parcels-Data-2006-thru-2017/9trm-uz8i) 

In [4]:
parcels_2016 = pd.read_csv('./data/2016_parcels.csv')

/Users/hunterowens/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2902: DtypeWarning: Columns (37) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
parcels_2016.head()

,ZIPcode,TaxRateArea_CITY,AIN,RollYear,TaxRateArea,AssessorID,PropertyLocation,PropertyType,PropertyUseCode,GeneralUseType,...,HouseFraction,StreetDirection,StreetName,UnitNo,City,ZIPcode5,rowID,CENTER_LAT,CENTER_LON,Location 1
0,,LOS ANGELES,2513002030,2016,16,2513-002-030,NaN,VAC,010V,Residential,...,,,,,,NaN,20162513002030,34.303785,-118.431512,"(34.30378543, -118.43151154)"
1,,BURBANK,2482012004,2016,2530,2482-012-004,NaN,VAC,010V,Residential,...,,,,,,NaN,20162482012004,34.154597,-118.340920,"(34.15459719, -118.34091962)"
2,,BURBANK,2485001014,2016,2530,2485-001-014,NaN,VAC,100V,Commercial,...,,,,,,NaN,20162485001014,34.152521,-118.348560,"(34.15252149, -118.34856035)"
3,,BURBANK,2485001015,2016,2530,2485-001-015,NaN,VAC,100V,Commercial,...,,,,,,NaN,20162485001015,34.152521,-118.348439,"(34.15252138, -118.34843850)"
4,,BURBANK,2485001016,2016,2530,2485-001-016,NaN,VAC,100V,Commercial,...,,,,,,NaN,20162485001016,34.152522,-118.348274,"(34.15252191, -118.34827448)"


## Percent of LA Count EFID Qualified 

In [6]:
pd.Series(parcels_2016.GeneralUseType.unique())

0       Residential
1        Commercial
2     Miscellaneous
3        Industrial
4     Institutional
5      Recreational
6          Dry Farm
7    Irrigated Farm
8         (missing)
dtype: object

In [7]:
parcels_2016['isUrban'] = parcels_2016.GeneralUseType.apply(lambda x: False if x == "Dry Farm" or x == "Irrigated Farm" else True)    

In [8]:
values = parcels_2016['isUrban'].value_counts()

In [12]:
percent_urban = round(((values[1] - values[0]) / parcels_2016['isUrban'].count()) * 100, 2)

{{percent_urban}} of the parcels in Los Angeles are EIFD eligble. However, this doesn't mean that certain combination of parcels wouldn't disqualify it for infill development. 

## How Much Tax Revenue could a total EFID Generate

We caculate the value of the increment via the below formula
 
**value of the increment** = [(predicted sales tax gained from the properties + predicted use tax gained from the properties) - (current sales tax gained from the properties + current use tax gained from the properties)] * number of years the increment district is valid for over the properties


* [2015-16 Property Tax Revenue for County](http://lacountyannualreport.com/budget/) - Property Taxes $5.682B
* [Tax Data](https://www.boe.ca.gov/DataPortal/SUTDRevDistCityCounty.aspx) 
* "Statewide property tax revenues are estimated to increase 5.9 percent in 2016‑17 and
5.3 percent in 2017‑18" - [State CAO estimates](http://www.ebudget.ca.gov/2017-18/pdf/BudgetSummary/RevenueEstimates.pdf) 
* "Statewise Sales Tax Revenue increased 2.9%" 

_Author's Note_: This work is missing some nuance. Notably, the growth curve and tax percentages aren't aligned with the EFID areas. Additionally, we do not subtract from the already allocated tax, ie Measure R & M in LA County, so this will be an overestimate. 

Additionally, the growth model is not fit at all. 

Finally, it assumes that we set the time of the Districts as 20 years starting in 2015, thanks to avaliable data. 

In [13]:
tax_data = pd.read_csv('./data/SUTDRevDistCityCounty.csv')

In [58]:
## Filter LA County 
filter_county = tax_data.County == "Los Angeles County"
filter_year = tax_data['Fiscal Year From'] == 2014

la_taxes_2015 = tax_data[filter_year & filter_county]

In [90]:
## Make Numeric
la_taxes_2015['revenue_numeric'] = la_taxes_2015['Revenue Distributed'].map(lambda x: x.replace("$", " ") \
                                                                                       .replace(",",""))
la_taxes_2015['revenue_numeric'] = pd.to_numeric(la_taxes_2015['revenue_numeric'])
total_sales_revenue_15 = la_taxes_2015['revenue_numeric'].sum()
total_prop_revenue_15 = 5682000000

/Users/hunterowens/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/hunterowens/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


        Total Sales Tax Revenue in Los Angeles due to Sales tax is {{"{:,}".format(total_sales_revenue_15)}}. Total Prop Tax Revenue is {{"{:,}".format(total_prop_revenue_15)}}

In [105]:
## Calculate Growth expected returns 
avg_sales_increase = .029
avg_prop_increase = .042

year_1 = 2015
year_2 = 2035

sales_revenue = dict()
prop_revenue = dict()
for year in range(year_1, year_2):
    collected_amt_sales = total_sales_revenue_15 * (1 + avg_sales_increase) ** (year - year_1)
    sales_revenue[year] = collected_amt_sales
    collected_amt_prop = total_prop_revenue_15 * (1 + avg_prop_increase) ** (year - year_1)
    prop_revenue[year] = collected_amt_prop

In [134]:
# Chart
line = Line(pd.Series(prop_revenue), title="Projected Property Tax Revenue Increase", ylabel='Dollars')
show(line)

In [138]:
# Capturable $$ 
df = pd.DataFrame([sales_revenue, prop_revenue]).T
df['sales_revenue'] = df[0]
df['prop_revenue'] = df[1]
df['capturable_prop'] = (df['prop_revenue'] - total_prop_revenue_15) * percent_urban
df['capturable_sales'] = (df['sales_revenue'] - total_sales_revenue_15) * percent_urban
df['total_capture'] = df.capturable_prop + df.capturable_sales
total_dollar_capture = df['total_capture'].sum()

In [137]:
show(Line(df['total_capture'], title="Total Capture"))

In total, a 20 year EFID for all possible areas in LA County would generate {{ "{:,}".format(total_dollar_capture) }}. Yes, that is __six trillion dollars__ over 20 years. 

## How much housing would that fund 
* percentage for affordable 
* percentage for local income 

Resources 

* [2014 State Cost Study](http://www.hcd.ca.gov/policy-research/plans-reports/docs/FinalAffordableHousingCostStudyReport-with-coverv2.pdf) 

In [139]:
# Discount for Bond Rate 

In [140]:
# Set asides for local income, affordable 